In [1]:
import pandas as pd
from dotenv import load_dotenv 
import sqlalchemy as sa
import os
from requests import HTTPError
import numpy as np
import json
import requests


In [2]:
def limpar_filme(dict_movie):
    try:
        dict_movie_limpo = {}
        dict_movie_limpo['Title'] = dict_movie['Title']
        dict_movie_limpo['rating_imdb'] = dict_movie['imdbRating']
        dict_movie_limpo['rating_mc'] = dict_movie['Metascore']
        dict_movie_limpo['imdb_id']= dict_movie['imdbID']
        dict_movie_limpo['imdb_votes']= dict_movie['imdbVotes']
        try:
            dict_movie_limpo['Revenue']= dict_movie['BoxOffice']
        except KeyError as e:
            dict_movie_limpo['Revenue'] = "n/a"
        return dict_movie_limpo
    except KeyError as e:
        print("filme com erro")
# [limpar_filme(filme) for filme in list_movies]

In [3]:
# rodados 13400
FILME_COMECO = 12400
FILME_FIM = 13400

In [4]:
load_dotenv('mysql.env')
url_banco = "localhost"
nome_db = "movie_db"
conn_str = f"mysql+pymysql://{os.getenv('MYSQL_USER')}:{os.getenv('MYSQL_PASS')}@{url_banco}/{nome_db}"
engine = sa.create_engine(conn_str)

In [5]:
df = pd.read_csv("movies_metadata.csv", dtype = "str", parse_dates = ["release_date"])

lista_id = list(df.imdb_id.values) 

mask = df["release_date"]>"2010-01-01"
df_2k10 = df[mask]

imdbid_2k10 = list(df_2k10.imdb_id.values)

In [6]:

list_movies = []
for i in imdbid_2k10[FILME_COMECO:FILME_FIM]:
    try:
        response = requests.get("https://www.omdbapi.com/?i={}&apikey=73324f26".format(i))
        response.raise_for_status()
        movie = response.json()
        list_movies.append(movie)
    except HTTPError as e:
        print(f'Falha na comunicação com o servidor:\n {e}')
        
    
dataFramemovies = pd.DataFrame([limpar_filme(filme) for filme in list_movies if limpar_filme(filme) is not None])
dataFramemovies.head()


dataFramemovies.to_sql('movies', engine, index = False, if_exists = 'append')    


333